In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
pd.read_csv('/content/drive/MyDrive/ML data set/weatherAUS.csv')

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [3]:
# prompt: preprocess the data set upto train test split

# Suppress warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ML data set/weatherAUS.csv')

# Display the first few rows
print(df.head())

# Display information about the dataset
print(df.info())

# Display descriptive statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Handle missing values (example: imputation for numerical columns, dropping for categorical)
# For demonstration, we'll fill numerical NaNs with the mean and drop rows with NaNs in critical categorical columns
numerical_cols = df.select_dtypes(include=np.number).columns
for col in numerical_cols:
  if df[col].isnull().any():
    df[col] = df[col].fillna(df[col].mean())

# Drop rows with NaNs in 'RainToday' and 'RainTomorrow' as these are target/important features
df.dropna(subset=['RainToday', 'RainTomorrow'], inplace=True)

# Handle categorical features (example: One-Hot Encoding)
categorical_cols = df.select_dtypes(include='object').columns
df = pd.get_dummies(df, columns=categorical_cols, dummy_na=False) # dummy_na=False to not create a column for NaN

# Convert the target variable 'RainTomorrow_Yes' to integer (1) and 'RainTomorrow_No' to integer (0)
# Assuming 'RainTomorrow' was a categorical column that got one-hot encoded
# If 'RainTomorrow' was the original target, convert it directly
if 'RainTomorrow_Yes' in df.columns:
    df['RainTomorrow'] = df['RainTomorrow_Yes'].astype(int)
    df.drop(['RainTomorrow_No', 'RainTomorrow_Yes'], axis=1, inplace=True)
elif 'RainTomorrow' in df.columns:
     # Map 'Yes' to 1 and 'No' to 0 if RainTomorrow was not one-hot encoded yet
     df['RainTomorrow'] = df['RainTomorrow'].map({'Yes': 1, 'No': 0})


# Define features (X) and target (y)
# Drop the original 'Date' column if it exists after one-hot encoding or as a datetime object
if 'Date' in df.columns:
    # If it's a datetime object, you might want to extract features or drop it
    if pd.api.types.is_datetime64_any_dtype(df['Date']):
        df.drop('Date', axis=1, inplace=True)
    # If it was one-hot encoded and became 'Date_YYYY-MM-DD' columns, handle as needed or drop
    # For simplicity, assuming we drop the original Date related columns
    date_cols = [col for col in df.columns if col.startswith('Date_')]
    df.drop(date_cols, axis=1, inplace=True)


# Separate features and target
if 'RainTomorrow' in df.columns:
    X = df.drop('RainTomorrow', axis=1)
    y = df['RainTomorrow']
else:
    print("Error: 'RainTomorrow' column not found after preprocessing.")
    # Handle the case where the target column is missing, e.g., exit or raise error
    exit()


# Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nTraining and testing data shapes:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1007.7    

In [5]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred = gnb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("\nModel Evaluation:")
print(f"Accuracy: {accuracy}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
class_report



Model Evaluation:
Accuracy: 0.5253569145535905

Confusion Matrix:
[[ 9949 11969]
 [ 1396  4844]]

Classification Report:


'              precision    recall  f1-score   support\n\n           0       0.88      0.45      0.60     21918\n           1       0.29      0.78      0.42      6240\n\n    accuracy                           0.53     28158\n   macro avg       0.58      0.62      0.51     28158\nweighted avg       0.75      0.53      0.56     28158\n'